In [85]:
%load_ext autoreload
%autoreload 2

import unicodecsv as csv
import datetime
import feedparser as fp
import newspaper
import helpers
from newspaper import Source, Article
import multiprocessing 
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
%%time
# Compiling papers with using rss information, producing newspaper objects:
all_cnn_papers = helpers.compile_papers(rss_feeds_dict=helpers.CNN_LINKS_DICT)
all_bbc_papers = helpers.compile_papers(rss_feeds_dict=helpers.BBC_LINKS_DICT)
all_fox_papers = helpers.compile_papers(rss_feeds_dict=helpers.FOX_LINKS_DICT)
all_rt_papers = helpers.compile_papers(rss_feeds_dict=helpers.RT_LINKS_DICT)
# No msnbc rss

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_world.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://rss.cnn.com/rss/cnn_topstories.rss
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://rss.cnn.com/rss/cnn_us.rss
[Parse lx

[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://feeds.foxnews.com/foxnews/health
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://feeds.foxnews.com/foxnews/most-popular
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://feeds.foxnews.com/foxnews/section/lifestyle
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Category parse ERR] http://feeds.foxnews.com/foxnews/most-popular
[Parse lxml ERR] Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
[Source parse ERR] http://feeds.foxnews.co

In [21]:
%%time
# Download articles, i.e. scrapeoff text and other data:
helpers.art_multithread_downloads(papers=all_cnn_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_bbc_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_fox_papers, threads_per_source=2)
helpers.art_multithread_downloads(papers=all_rt_papers, threads_per_source=2)

CPU times: user 37.9 s, sys: 6.41 s, total: 44.3 s
Wall time: 2min 47s


In [37]:
master_paper_list = [all_cnn_papers, 
                     all_bbc_papers,
                     all_fox_papers,
                     all_rt_papers]

In [38]:
%%time
# Parse all articles:
for paper_list in master_paper_list:
    for paper in paper_list:
        paper.parse_articles()

/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 124. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41486
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 13 bytes but only got 0. Skipping tag 42033
  " Skipping tag %s" % (si

/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 144. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 148. Skipping tag 37510
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 8. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag))
/Users/alexander/anaconda3/envs/NewsBiasPython2/lib/python2.7/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 4. Skipping tag 37378
  " Skipping tag %s" %

CPU times: user 22min 40s, sys: 19.3 s, total: 22min 59s
Wall time: 31min 18s


In [78]:
def write_out_articles(newspaper_obj, filepath, writeout_type='conventional'):
    """
    TODO: finish this docstring
    :param newspaper_obj:
    :param filepath: str, filepath, WITHOUT the filename
    :param writeout_type: str, 'custom' or 'conventional'
    :return:
    """
    now = datetime.datetime.now()
    current_date = '{}_{}_{}'.format(str(now.year),
                                     str(now.month),
                                     str(now.day))
    if writeout_type == 'custom':
        csv_name = filepath + current_date + '_' + newspaper_obj.description \
                   + '.csv'
    else:
        csv_name = filepath + current_date + '_' + newspaper_obj.brand + '.csv'
    with open(csv_name, 'w') as outcsv:
        writer = csv.writer(outcsv, encoding='utf-8')
        if writeout_type == 'conventional':
            header = ['source_url', 'url', 'title', 'movies', 'text',
                      'keywords', 'meta_keywords', 'tags', 'authors',
                      'publish_date', 'summary', 'html', 'article_html',
                      'meta_description', 'meta_data', 'canonical_link']
        else:
            header = ['source_url', 'url', 'title', 'movies', 'text',
                             'keywords', 'meta_keywords', 'tags', 'authors',
                             'publish_date', 'summary', 'html', 'article_html',
                             'meta_description', 'meta_data', 'canonical_link',
                             'rss_title', 'rss_link', 'rss_id', 'rss_published',
                             'rss_published_parsed', 'rss_feedburner_origlink',
                             'paper_section_name']
        writer.writerow(header)
        for article in newspaper_obj.articles:
            source_url = article.source_url
            url = article.url
            title = article.title
            movies = article.movies
            text = article.text
            keywords = article.keywords
            meta_keywords = article.meta_keywords
            tags = article.tags
            authors = article.authors
            publish_date = article.published_date
            summary = article.summary
            html = article.html
            article_html = article.article_html
            meta_description = article.meta_description
            meta_data = article.meta_data
            canonical_link = article.canonical_link
            if writeout_type == 'custom':
                rss_title = article.additional_data['rss_title']
                rss_link = article.additional_data['rss_link']
                rss_id = article.additional_data['rss_id']
                rss_published = article.additional_data['rss_published']
                rss_published_parsed = \
                    article.additional_data['rss_published_parsed']
                rss_feedburner_origlink = \
                    article.additional_data['rss_feedburner_origlink']
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link, rss_title, rss_link,
                            rss_id, rss_published, rss_published_parsed,
                            rss_feedburner_origlink,
                            newspaper_obj.description]
            else:
                data_row = [source_url, url, title, movies, text, keywords,
                            meta_keywords, tags, authors, publish_date,
                            summary, html, article_html, meta_description,
                            meta_data, canonical_link]
            writer.writerow(data_row)

In [79]:
filepath = 'data/custom_data_pulls/'

In [80]:
%%time
for paper_list in master_paper_list:
    for paper in paper_list:
        write_out_articles(newspaper_obj=paper,
                                   filepath=filepath, 
                                   writeout_type='custom')

CPU times: user 10.7 s, sys: 4.07 s, total: 14.7 s
Wall time: 27 s


In [87]:
df = pd.read_csv('data/custom_data_pulls/2019_1_31_fox_us.csv')

In [92]:
df.rss_link

0     http://feeds.foxnews.com/~r/foxnews/national/~...
1     http://feeds.foxnews.com/~r/foxnews/national/~...
2     http://feeds.foxnews.com/~r/foxnews/national/~...
3     http://feeds.foxnews.com/~r/foxnews/national/~...
4     http://feeds.foxnews.com/~r/foxnews/national/~...
5     http://feeds.foxnews.com/~r/foxnews/national/~...
6     http://feeds.foxnews.com/~r/foxnews/national/~...
7     http://feeds.foxnews.com/~r/foxnews/national/~...
8     http://feeds.foxnews.com/~r/foxnews/national/~...
9     http://feeds.foxnews.com/~r/foxnews/national/~...
10    http://feeds.foxnews.com/~r/foxnews/national/~...
11    http://feeds.foxnews.com/~r/foxnews/national/~...
12    http://feeds.foxnews.com/~r/foxnews/national/~...
13    http://feeds.foxnews.com/~r/foxnews/national/~...
14    http://feeds.foxnews.com/~r/foxnews/national/~...
15    http://feeds.foxnews.com/~r/foxnews/national/~...
16    http://feeds.foxnews.com/~r/foxnews/national/~...
17    http://feeds.foxnews.com/~r/foxnews/nation